In [3]:
# STEP 1 - Set up Azure ML workspace
import json
from azureml.core import Workspace, Environment
from azureml.core.model import Model
from azureml.core.environment import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

my_id = "_____"

ws = Workspace.get(
    name="workspace",
    subscription_id=my_id,
    resource_group="resource_group2"
)

# registrar el modelo
mname = "model"
registered_model = Model.register(
    model_path="model_bankruptcy.pkl",
    model_name=mname,
    workspace=ws
)

# cargar umbral 
with open("umbral_bankruptcy.json", "r") as umb:
    umbral = json.load(umb)["umbral"]

# Crear archivo score.py
scorepy = f"""
import json
import numpy as np
import pandas as pd
import joblib
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('{mname}')
    model = joblib.load(model_path)  # (xgb_model, logistic_model)
    global xgb_model, logistic_model
    xgb_model, logistic_model = model

def run(raw_data):
    try:
        data = json.loads(raw_data)['data'][0]
        df = pd.DataFrame(data)

        # Obtener probabilidades de XGBoost
        xgb_probs = xgb_model.predict_proba(df)[:, 1].reshape(-1, 1)

        # Aplicar regresión logística sobre las probabilidades
        final_probs = logistic_model.predict_proba(xgb_probs)[:, 1]

        # Aplicar umbral dinámico
        umbral = {umbral}
        final_preds = [1 if p > umbral else 0 for p in final_probs]

        return json.dumps(final_preds)

    except Exception as e:
        return json.dumps(str(e))
"""

with open("score.py", "w") as f:
    f.write(scorepy)

# STEP 2 - Configuración del entorno
virtual_env = Environment("env-bankruptcy-model")
virtual_env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=["pandas", "scikit-learn", "xgboost", "numpy", "joblib"]
)

inference_config = InferenceConfig(
    environment=virtual_env,
    entry_script="score.py"
)

aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

# STEP 3 - Desplegar el servicio
service = Model.deploy(
    workspace=ws,
    name="service-bankruptcy",
    models=[registered_model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)

service.wait_for_deployment(show_output=True)

# Guardar URI del servicio
scoring_uri = service.scoring_uri
with open("uri.json", "w") as f:
    json.dump({"URI": [scoring_uri]}, f)

print("Despliegue exitoso.")
print("URI del endpoint:", scoring_uri)

Registering model model


/var/folders/8y/0g_q78s11fs7byv7zy57g1v40000gn/T/ipykernel_4358/788481890.py:82: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-03-31 17:43:03-06:00 Creating Container Registry if not exists..
2025-03-31 17:53:04-06:00 Registering the environment.
2025-03-31 17:53:06-06:00 Building image..
2025-03-31 18:02:19-06:00 Generating deployment configuration.
2025-03-31 18:02:20-06:00 Submitting deployment to compute..
2025-03-31 18:02:29-06:00 Checking the status of deployment service-bankruptcy..
2025-03-31 18:03:36-06:00 Checking the status of inference endpoint service-bankruptcy.
Succeeded
ACI service creation operation finished, operation "Succeeded"
✅ Despliegue exitoso.
🔗 URI del endpoint: http://0ad22dee-50c4-4abd-afde-2831bd2566a4.centralindia.azurecontainer.io/score
